When ip addresses are added to the mywhois database there is a process that checks each new cidr for children that may already been in the database. When some are found, the parent is subdivided into a list of cidrs that exclude those addresses. This list is then added to the database.

In order to check that the calculation is accurate, a dict, risk.families, is made. Its key is the cidr of the original parent. The value stored is a list of 2 lists, children, the list of the children of the original parent, and parents, the list of the subdivided parent that exclude the children.

After the load program was complete the families object was saved to disk as follows:
1. Convert dict to families\[cidr\]={ "children": \[...\], "parents": \[...\]}
2. Convert to json
3. Write to file 

```# Construct a dict, cidrs, from the families dict as strings.
cidrs={}
for k,v in newip.risk.families.items():
    cidrs[str(k)] = {'children': [str(x) for x in v[0]], 
                     'parents':  [str(x) for x in v[1]]}

# Change the dict into a json list, cidrout, so it is all plain text
import json
cidrout = json.dumps(cidrs)

# And save it on a file 'families.json'
with open('families.json', 'w') as ff:
    ff.write(cidrout)
```

The program reads the saved families, with string objects, and verifies that there are no gaps when the subdivided parent is added to the database. That they are all, when combined with their children, contiguous.Two cidrs are contiguous when the last host of the previous cidr is is 3 less than the first host of the next cidr. The difference is 3 because some of the cidr addresses are not used.

In [1]:
from ipaddress import *
import json

def read_families(fn='families.json'):
    # read the json that was stored in a file
    with open(fn, 'r') as hi:
        j_families = hi.read()
    families = json.loads(j_families)
    return families

def edges(lst):
    # Find list of tuples (firstip, lastip) of a list of cidrs. 
    # All objects are converted to ip_network objects
    edges = lambda x: (list(ip_network(x).hosts())[0], list(ip_network(x).hosts())[-1])
    r = []
    for l in lst:
        r.append(edges(l))
    return r

def prt_gaps(s):
    # Output any gap. s is sorted list of children + subdivided parent
    ns = len(s)
    for i in range(0, ns-1):
        diff = s[i+1][0]==s[i][1]+3
        if not diff:
            print(f'{i=:<12} {str(s[i+1][0]):20} {str(s[i][1]):20} {diff}')

def merge_members(parent):
    # Merge the children and subdivided parents of the original parent
    l1 = parent['children']
    l2 = parent['parents']
    return edges(l1 + l2)

In [2]:

families = read_families()

print('Be patient it takes time')
n = len(families)
c = 0
for cidr, parent in families.items():
    # Look over all the families. 
    # Provide progress reports to user
    c += 1
    ctr = f'{c} of {n}'
    print(f'{ctr:15} {str(cidr)}')
    
    s = merge_members(parent)
    s = sorted(s, key=lambda x: x[0])
    prt_gaps(s)

Be patient it takes time
1 of 86         24.44.0.0/14
2 of 86         151.200.0.0/14
3 of 86         68.160.0.0/14
4 of 86         69.112.0.0/12
5 of 86         50.74.0.0/15
6 of 86         47.21.0.0/17
7 of 86         173.2.0.0/15
8 of 86         72.43.0.0/16
9 of 86         68.192.0.0/13
10 of 86        206.217.128.0/20
11 of 86        24.89.144.0/20
12 of 86        73.0.0.0/8
13 of 86        154.0.0.0/8
14 of 86        208.125.0.0/16
15 of 86        24.188.0.0/14
16 of 86        198.179.64.0/18
17 of 86        76.78.0.0/16
18 of 86        75.128.0.0/12
19 of 86        108.58.0.0/16
20 of 86        144.121.0.0/16
21 of 86        12.0.0.0/8
22 of 86        75.127.240.0/21
23 of 86        24.30.224.0/19
24 of 86        108.176.0.0/17
25 of 86        72.0.128.0/19
26 of 86        64.128.0.0/15
27 of 86        47.21.224.0/20
28 of 86        160.72.0.0/16
29 of 86        24.97.0.0/16
30 of 86        69.164.128.0/20
31 of 86        108.170.64.0/18
32 of 86        75.99.0.0/18
33 of 86     